# Fast Food

# The goal here:
- trying to find the healthier fast food options that would be a great dinner option for those who are tired after work and want a meal that will fit within the standard 2,000 calorie limit (generalized calorie amount, calorie needs will vary between every individual)
- considering the following macros: carbs, fats, and protein. The macros division can be split up in varying ways, but I will keep the standard 50% carbs, 30% protein, and 20% fat. I would also want to emphasize sodium and fiber. The recommended amount of sodium is 2,300 mg a day, and the average amount of fiber for female and male is 21 to 30 (lower scale). 
- Ideally, if you had a balanced breakfast, lunch, and maybe a few snacks, the calorie range for dinner would be around 500-600 calories. 1 gram of carb is 4 calories, 1 gram of protein is 4 calories, and 1 gram of fat is 9 calories. Taking that into consideration, the distribution of 600 calories for the macros would be 75 grams of carbs, 45 gram of protein, and 13 grams of fat.

In [1]:
# Setting up
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Connect to resource file
fast_food_path = "resources/fastfood.csv"

# Read file shown above
fast_food_data = pd.read_csv(fast_food_path)

# Display data
fast_food_data.head()

,restaurant,item,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein,vit_a,vit_c,calcium,salad
0,Mcdonalds,Artisan Grilled Chicken Sandwich,380,60,7,2.0,0.0,95,1110,44,3.0,11,37.0,4.0,20.0,20.0,Other
1,Mcdonalds,Single Bacon Smokehouse Burger,840,410,45,17.0,1.5,130,1580,62,2.0,18,46.0,6.0,20.0,20.0,Other
2,Mcdonalds,Double Bacon Smokehouse Burger,1130,600,67,27.0,3.0,220,1920,63,3.0,18,70.0,10.0,20.0,50.0,Other
3,Mcdonalds,Grilled Bacon Smokehouse Chicken Sandwich,750,280,31,10.0,0.5,155,1940,62,2.0,18,55.0,6.0,25.0,20.0,Other
4,Mcdonalds,Crispy Bacon Smokehouse Chicken Sandwich,920,410,45,12.0,0.5,120,1980,81,4.0,18,46.0,6.0,20.0,20.0,Other
